# socket.io Adapter
https://socket.io/docs/v4/mongo-adapter/
- 다른 서버들 사이에 실시간 어플리케이션 동기화

- Adapter는 누가 연결되었는지, 현재 어플리케이션에 room이 얼마나 있는지 알려줌

sids에는 개인방, rooms에는 개인방,공개방 다있음

rooms가 sids를 포함한다 보면됨

공개방만 얻고 싶을때는 rooms에서 sids를 빼면 됨

MAP과 adapter

socket.io에서 room과 sids 과 자동적으로 생성

### server.js
```javascript
import express from "express";
import http from "http";
import SocketIO from "socket.io";

const app = express();
const port = 3000;

// json file 입출력 설정
app.use(express.json());

// pug 설정
app.set("view engine", "pug");
app.set("views", __dirname + "/views");

// static 작업
app.use("/public", express.static(__dirname + "/public"));

// route 설정
app.get("/", (_, res) => res.render("home"));
app.get("/*", (_, res) => res.redirect("/"));

// http server
const httpServer = http.createServer(app);

// io server
const wsServer = SocketIO(httpServer);

function publicRooms() {
  const {
    sockets: {
      adapter: { sids, rooms },
    },
  } = wsServer;

  const publicRooms = [];
  rooms.forEach((_, key) => {
    if (sids.get(key) === undefined) {
      publicRooms.push(key);
    }
  });
  // sids에는 개인방, rooms에는 개인방,공개방 다있음
  // rooms가 sids를 포함한다 보면됨
  // 공개방만 얻고 싶을때는 rooms에서 sids를 빼면 됨
  return publicRooms;
}

wsServer.on("connection", (socket) => {
  socket["nickname"] = "Anon";
  socket.onAny((event) => {
    console.log(wsServer.sockets.adapter);
    console.log(`Socket Event: ${event}`);
  });

  socket.on("enter_room", (roomName, done) => {
    socket.join(roomName);
    done();
    // room 전체에 메시지 보내기
    socket.to(roomName).emit("welcome", socket.nickname);
  });

  socket.on("disconnecting", () => {
    socket.rooms.forEach((room) =>
      socket.to(room).emit("bye", socket.nickname)
    );
  });

  socket.on("message", (msg, room, done) => {
    socket.to(room).emit("message", `${socket.nickname} : ${msg}`);
    done();
  });

  socket.on("nickname", (nickname) => (socket["nickname"] = nickname));
});

httpServer.listen(port, () => {
  console.log(`Joom app listening on http://localhost:${port}`);
});

```